# Regression: Feature Selection and LASSO (Interpretation)

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data
Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('C:\Users\kc_house_data.gl')

This non-commercial license of GraphLab Create for academic use is assigned to bhises2@mcmaster.ca and will expire on December 08, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\hp\AppData\Local\Temp\graphlab_server_1518223940.log.0


# Create new features
As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before creating a new feature.
sales['floors'] = sales['floors'].astype(int) 
sales['floors_square'] = sales['floors']*sales['floors']

# Learn regression weights with L1 penalty
Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (l1_penalty) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.) Note that it's important to set l2_penalty=0 to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.659107     | 6877408.472440     | 386444.117469 |

| 6         | 7        | 0.000002  | 1.697136     | 6848120.275355     | 380577.022481 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:

x = model_all.get("coefficients")
x.print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274952.62044 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8483.95148798 |  None  |
|   sqft_living    |  None | 24.4238022551 |  None  |
| sqft_living_sqrt |  None | 351.097833343 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 850.427363977 |  None  |
|    sqft_above    |  None | 20.0777654516 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection.

QUIZ QUESTION: According to this list of weights, which of the features have been chosen?

grade, sqft_living

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:

Split our sales data into 2 sets: training and test
Further split our training data into two sets: train, validation
Be very careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:

For l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type np.logspace(1, 7, num=13).)
Fit a regression model with a given l1_penalty on TRAIN data. Specify l1_penalty=l1_penalty and l2_penalty=0. in the parameter list.
Compute the RSS on VALIDATION data (here you will want to use .predict()) for that l1_penalty
Report which l1_penalty produced the lowest RSS on validation data.
When you call linear_regression.create() make sure you set validation_set = None.

Note: you can turn off the print out of linear_regression.create() with verbose = False

In [8]:
import numpy as np

In [9]:
min_rss = None
min_l1_penalty = 0.0
best_model = None

for l1 in np.logspace(1,7,num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False, l2_penalty=0., l1_penalty=l1)
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = (residuals ** 2).sum()
    
    if (min_rss is None or rss <= min_rss):
        min_rss = rss
        min_l1_penalty = l1
        best_model = model
        
print (min_l1_penalty, "%.1e" % min_rss)

(10.0, '6.3e+14')


In [10]:
test_predictions = best_model.predict(testing)
residuals = testing['price'] - test_predictions
rss = (residuals ** 2).sum()
print rss

1.57678098725e+14


In [11]:
(best_model['coefficients']['value']).nnz()

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:

Explore a large range of l1_penalty values to find a narrow region of l1_penalty values where models are likely to have the desired number of non-zero weights.
Further explore the narrow region you found to find a good value for l1_penalty that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for l1_penalty.

In [12]:
max_nonzeros = 7

# Exploring the larger range of values to find a narrow range with the desired sparsity
Let's define a wide range of possible l1_penalty_values:

In [13]:

l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible l1_penalty values:

For l1_penalty in np.logspace(8, 10, num=20):
Fit a regression model with a given l1_penalty on TRAIN data. Specify l1_penalty=l1_penalty and l2_penalty=0. in the parameter list. When you call linear_regression.create() make sure you set validation_set = None
Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
Hint: model['coefficients']['value'] gives you an SArray with the parameters you learned. If you call the method .nnz() on it, you will find the number of non-zero parameters!

In [14]:
num_coefficients = []
l1_penalty_min = None
l1_penalty_max = None
for l1 in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False, l2_penalty=0., l1_penalty=l1)
    num_non_zeroes = (model['coefficients']['value']).nnz()
    num_coefficients.append(num_non_zeroes)
    print (l1, num_non_zeroes)
    if (num_non_zeroes > max_nonzeros and (l1_penalty_min is None or l1 > l1_penalty_min)):
        l1_penalty_min = l1
    elif (num_non_zeroes < max_nonzeros and (l1_penalty_max is None or l1 > l1_penalty_max)):
        l1_penalty_max = l1
        break
    
print (l1_penalty_min, l1_penalty_max)

(100000000.0, 18)
(127427498.57031322, 18)
(162377673.91887242, 18)
(206913808.11147901, 18)
(263665089.87303555, 17)
(335981828.62837881, 17)
(428133239.8719396, 17)
(545559478.11685145, 17)
(695192796.17755914, 17)
(885866790.41008317, 16)
(1128837891.6846883, 15)
(1438449888.2876658, 15)
(1832980710.8324375, 13)
(2335721469.0901213, 11)
(2976351441.6313128, 10)
(3792690190.7322536, 6)
(2976351441.6313128, 3792690190.7322536)


Out of this large range, we want to find the two ends of our desired narrow range of l1_penalty. At one end, we will have l1_penalty values that have too few non-zeros, and at the other end, we will have an l1_penalty that has too many non-zeros.

More formally, find:

The largest l1_penalty that has more non-zeros than max_nonzero (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
Store this value in the variable l1_penalty_min (we will use it later)
The smallest l1_penalty that has fewer non-zeros than max_nonzero (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
Store this value in the variable l1_penalty_max (we will use it later)
Hint: there are many ways to do this, e.g.:

Programmatically within the loop above
Creating a list with the number of non-zeros for each value of l1_penalty and inspecting it to find the appropriate boundaries.

In [15]:
l1_penalty_min = 2976351441.6313133
l1_penalty_max = 3792690190.7322536

Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set
We will now explore the narrow region of l1_penalty values we found:

In [16]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
Fit a regression model with a given l1_penalty on TRAIN data. Specify l1_penalty=l1_penalty and l2_penalty=0. in the parameter list. When you call linear_regression.create() make sure you set validation_set = None
Measure the RSS of the learned model on the VALIDATION set
Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to max_nonzero

In [18]:
min_rss = None
best_model = None
best_penalty = 0.0
for l1 in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False, l2_penalty=0., l1_penalty=l1)
    
    num_non_zeroes = (model['coefficients']['value']).nnz()
    predictions = model.predict(validation)
    rss = ((validation['price'] - predictions) ** 2).sum()
    
    if (num_non_zeroes == max_nonzeros and (min_rss is None or rss < min_rss)):
        min_rss = rss
        best_penalty = l1
        best_model = model
        
print(best_penalty, min_rss)

(3320073020.2001305, 1029979472296642.9)


In [19]:
(model.get("coefficients")).print_rows(num_rows = 18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 240613.124878 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 13959.9330759 |  None  |
|   sqft_living    |  None | 30.6553907037 |  None  |
| sqft_living_sqrt |  None | 598.158171599 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2303.49748594 |  None  |
|    sqft_above    |  None | 27.6805049159 |  None  |
|  sqft_basement   |  None |